In [ ]:
!pip install wandb

  Using cached wandb-0.16.1-py3-none-any.whl (2.1 MB)
  Using cached GitPython-3.1.40-py3-none-any.whl (190 kB)
  Using cached sentry_sdk-1.39.1-py2.py3-none-any.whl (254 kB)
  Using cached docker_pycreds-0.4.0-py2.py3-none-any.whl (9.0 kB)
  Using cached setproctitle-1.3.3-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (30 kB)
  Using cached gitdb-4.0.11-py3-none-any.whl (62 kB)
  Using cached smmap-5.0.1-py3-none-any.whl (24 kB)


In [ ]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
import wandb

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F

In [ ]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN,self).__init__()
    #convolution
    self.conv1=nn.Conv2d(in_channels=3,out_channels=32,kernel_size=(3,3),padding=1)
    self.conv2=nn.Conv2d(in_channels=32,out_channels=64,kernel_size=(3,3),padding=1)
    #pooling
    self.pool=nn.MaxPool2d(kernel_size=(2,2))

    self.fc1=nn.Linear(64*7*7,128)
    self.fc2=nn.Linear(128,10)

  def forward(self,x):
    # 각자 합성곱, 활성함수, pooling 적용
    x=self.pool(F.relu(self.conv1(x)))
    x=self.pool(F.relu(self.conv2(x)))
    x=x.view(-1,64*7*7)
    x=F.relu(self.fc1(x))
    x=self.fc2(x)

    return F.softmax(x,dim=1)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# 앞서 정의한 모델을 장치로 올림, 모델 정의
model = CNN().to(device)

In [ ]:
import torch.optim as optim

# 모델 학습을 위한 손실 함수와 최적화 함수
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [ ]:
# 압축 풀기용 코드 한번만 실행
'''
import zipfile
zipfile.ZipFile('drive/My Drive/openAI/data/data.zip').extractall('drive/My Drive/openAI/data')
'''

"\nimport zipfile\nzipfile.ZipFile('drive/My Drive/openAI/data/data.zip').extractall('drive/My Drive/openAI/data')\n"

In [ ]:
import os
import random
from PIL import Image, UnidentifiedImageError,ImageFile

In [ ]:
# 원본 이미지 검증
'''
def validate_image(filepath):
    try:
        img = Image.open(filepath).convert('RGB')
        img.load()
    except UnidentifiedImageError:
        print(f'Corrupted Image is found at: {filepath}')
        return False
    except (IOError, OSError):
        print(f'Truncated Image is found at: {filepath}')
        return False
    else:
        return True
'''

"\ndef validate_image(filepath):\n    try:\n        img = Image.open(filepath).convert('RGB')\n        img.load()\n    except UnidentifiedImageError:\n        print(f'Corrupted Image is found at: {filepath}')\n        return False\n    except (IOError, OSError):\n        print(f'Truncated Image is found at: {filepath}')\n        return False\n    else:\n        return True\n"

In [ ]:
# 이미지 검증
'''
root = 'drive/My Drive/openAI/data/유증상_라벨_검증'

dirs = os.listdir(root)

for dir_ in dirs:
    folder_path = os.path.join(root, dir_)
    files = os.listdir(folder_path)

    for file_name in files:
        file_path = os.path.join(folder_path, file_name)

        # 파일 확장자 확인
        _, file_extension = os.path.splitext(file_name)

        if file_extension.lower() == '.jpg':
            valid = validate_image(file_path)
            if not valid:
                # 오류가 있는 이미지 파일 삭제
                os.remove(file_path)
        elif file_extension.lower() == '.json':
            # JSON 파일은 삭제하지 않음
            pass
'''

"\nroot = 'drive/My Drive/openAI/data/유증상_라벨_검증'\n\ndirs = os.listdir(root)\n\nfor dir_ in dirs:\n    folder_path = os.path.join(root, dir_)\n    files = os.listdir(folder_path)\n\n    for file_name in files:\n        file_path = os.path.join(folder_path, file_name)\n\n        # 파일 확장자 확인\n        _, file_extension = os.path.splitext(file_name)\n\n        if file_extension.lower() == '.jpg':\n            valid = validate_image(file_path)\n            if not valid:\n                # 오류가 있는 이미지 파일 삭제\n                os.remove(file_path)\n        elif file_extension.lower() == '.json':\n            # JSON 파일은 삭제하지 않음\n            pass\n"

In [ ]:
# json 파일에서 뽑아 사용하는 경우
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
import json
class MyDataLoader(Dataset):
    def __init__(self, root_folder, batch_size, transform):
        self.root_folder = root_folder
        self.transform = transform
        self.batch_size = batch_size
        self.data = self.load_data()

    def load_data(self):
        data = []

        for root, dirs, files in os.walk(self.root_folder):
            for filename in files:
                if filename.endswith('.jpg'):
                    img_path = os.path.join(root, filename)

                    json_filename = os.path.splitext(filename)[0] + '.json'
                    json_file_path = os.path.join(root, json_filename)

                    if not os.path.exists(json_file_path):
                        continue

                    with open(json_file_path, 'r') as json_file:
                        metadata = json.load(json_file)

                    label = metadata['metaData']['lesions']
                    data.append((img_path, label))

        return data

    def __len__(self):
        return len(self.data)

    def label_convert(self,label):
        label_mapping = {"A2": 0, "A3": 1, "A4": 2, "A5": 3, "A6": 4}
        return label_mapping.get(label,-1)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        label=self.label_convert(label)

        return image, label


In [ ]:
#이미지 데이터 가져오기
from torchvision import transforms
'''
#데이터 위치 지정
train_data = torchvision.datasets.ImageFolder(
    root = 'drive/My Drive/openAI/data/유증상_라벨_학습',
    transform= transforms.Compose([
      # 사이즈가 너무 커서 세션 다운되므로 줄임
      transforms.Resize([28,28]),
      # dataloader가 PIL 이미지를 수신하기 위해 이미지의 변형이 필요
      transforms.ToTensor()
    ])
)
test_data= torchvision.datasets.ImageFolder(
    root = 'drive/My Drive/openAI/data/유증상_라벨_검증',
    transform= transforms.Compose([
      transforms.Resize([28,28]),
      transforms.ToTensor()
    ])
)
print(train_data)
print(test_data)

#데이터를 배치 사이즈에 맞게 가져오기
#배치 사이즈=100

train_loader=MyDataLoader(train_data,batch_size=100)
test_loader=MyDataLoader(test_data,batch_size=100)
'''

train_root='drive/My Drive/openAI/data/유증상_라벨_학습'
test_root= 'drive/My Drive/openAI/data/유증상_라벨_검증'

transform=transforms.Compose([
    transforms.Resize([28, 28]),
    transforms.ToTensor()
])

train_loader = MyDataLoader(train_root, batch_size=100, transform=transform)

test_loader = MyDataLoader(test_root, batch_size=100, transform=transform)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
print(train_loader[0][0])
print(train_loader[0][1])

tensor([[[0.8980, 0.8941, 0.8824,  ..., 0.3176, 0.2588, 0.2863],
         [0.8745, 0.8784, 0.8667,  ..., 0.6706, 0.4510, 0.3333],
         [0.8510, 0.8471, 0.8431,  ..., 0.8784, 0.8078, 0.6353],
         ...,
         [0.4980, 0.4902, 0.4784,  ..., 0.7255, 0.7725, 0.7412],
         [0.4980, 0.4784, 0.4588,  ..., 0.7020, 0.7608, 0.7451],
         [0.4706, 0.4510, 0.4314,  ..., 0.6353, 0.7529, 0.7608]],

        [[0.9020, 0.8902, 0.8706,  ..., 0.2118, 0.1373, 0.1255],
         [0.8667, 0.8667, 0.8549,  ..., 0.6353, 0.3725, 0.1686],
         [0.8314, 0.8235, 0.8196,  ..., 0.8902, 0.8078, 0.5882],
         ...,
         [0.4902, 0.4784, 0.4667,  ..., 0.7216, 0.7725, 0.7412],
         [0.4824, 0.4627, 0.4431,  ..., 0.6824, 0.7451, 0.7412],
         [0.4510, 0.4314, 0.4078,  ..., 0.5804, 0.7176, 0.7490]],

        [[0.8824, 0.8706, 0.8510,  ..., 0.1490, 0.0706, 0.0431],
         [0.8549, 0.8510, 0.8353,  ..., 0.6000, 0.3255, 0.0784],
         [0.8157, 0.8039, 0.7961,  ..., 0.8745, 0.7922, 0.

In [ ]:
wandb.init(project='openSourceAI',name='CNN')

wandb: Currently logged in as: euiyang2000. Use `wandb login --relogin` to force relogin


In [ ]:
#학습
for epoch in range(5):
  model.train()
  running_loss = 0.0
  correct = 0
  total = 0
  for i, data in enumerate(train_loader, 0):
      # Get the inputs; data is a list of [inputs, labels]
      #label 데이터 추출 후 담기
      inputs, labels= data

      if isinstance(labels, str):
        labels = torch.tensor([int(labels)]).to(device)

      inputs, labels= inputs.to(device), labels.to(device)

      # Zero the parameter gradients
      optimizer.zero_grad()

      # Forward pass
      outputs = model(inputs)

      # Compute the loss
      loss = criterion(outputs, labels)

      # Backward pass and optimize
      loss.backward()
      optimizer.step()

      # Accuracy
      _, predicted = torch.max(outputs.data, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()

      # log loss
      wandb.log({'train/loss':loss.item(), 'train/acc':correct/total})

      # Print statistics
      running_loss += loss.item()
      if i % 100 == 99:    # Print every 100 mini-batches
          print(f'Epoch {epoch + 1}, Batch {i + 1}, Loss: {running_loss / 100}')
          running_loss = 0.0

  model.eval()
  with torch.no_grad():
    running_loss = 0.0
    correct = 0
    total = 0
    for i, data in enumerate(test_loader, 0):
      #label 데이터 추출 후 담기
      inputs, labels= data

      if isinstance(labels, str):
        labels = torch.tensor([int(labels)]).to(device)

      inputs, labels= inputs.to(device), labels.to(device)

      # Forward pass
      outputs = model(inputs)

      # Compute the loss
      loss = criterion(outputs, labels)

      # Accuracy
      _, predicted = torch.max(outputs.data, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()
      running_loss += loss.item()

      # log loss
      wandb.log({'test/loss':loss.item(), 'test/acc':correct/total})

      # Print statistics
      running_loss += loss.item()
      if i % 100 == 99:    # Print every 100 mini-batches
          print(f'Test Epoch {epoch + 1}, Batch {i + 1}, Loss: {running_loss / 100}')
          running_loss = 0.0


print('Finished Training')
wandb.finish()

AttributeError: ignored

In [ ]:
# 배치정규화 CNN
class CNN(nn.Module):
  def __init__(self):
      super(CNN, self).__init__()
      self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=(3, 3), padding=1)
      self.bn1 = nn.BatchNorm2d(32)
      self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3), padding=1)
      self.bn2 = nn.BatchNorm2d(64)
      self.pool = nn.MaxPool2d(kernel_size=(2, 2))
      self.fc1 = nn.Linear(64 * 7 * 7, 128)
      self.fc2 = nn.Linear(128, 10)

  def forward(self,x):
    # 각자 합성곱, 활성함수, pooling 적용
    x=self.pool(F.relu(self.conv1(x)))
    x=self.pool(F.relu(self.conv2(x)))
    x=x.view(-1,64*7*7)
    x=F.relu(self.fc1(x))
    x=self.fc2(x)

    return F.softmax(x,dim=1)

In [1]:
# 배치정규화+드롭아웃 CNN
class CNN(nn.Module):
  def __init__(self):
      super(CNN, self).__init__()
      self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=(3, 3), padding=1)
      self.bn1 = nn.BatchNorm2d(32)
      self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3), padding=1)
      self.bn2 = nn.BatchNorm2d(64)
      self.pool = nn.MaxPool2d(kernel_size=(2, 2))
      self.fc1 = nn.Linear(64 * 7 * 7, 128)
      self.dropout = nn.Dropout(0.5)
      self.fc2 = nn.Linear(128, 10)

  def forward(self,x):
    # 각자 합성곱, 활성함수, pooling 적용
    x=self.pool(F.relu(self.conv1(x)))
    x=self.pool(F.relu(self.conv2(x)))
    x=x.view(-1,64*7*7)
    x=F.relu(self.fc1(x))
    x=self.fc2(x)

    return F.softmax(x,dim=1)

NameError: ignored